In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import pickle
from collections import OrderedDict
import copy
from scipy.sparse import csr_matrix
from scipy import io
import seaborn as sns
import joblib
# from base import *
from joblib import Parallel, delayed
import random
import scipy

In [3]:
MNM_nb_folder = os.path.join('..', '..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..', 'pylib')
sys.path.append(python_lib_folder)

In [4]:
from MNMAPI import *
from MNM_mcnb import *
from mcDODE import *

In [5]:
data_folder = os.path.join('/home/lemma/Documents/MAC-POSTS/data/input_files_7link_multiclass')

In [6]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)

In [7]:
observed_link_list = [3, 4, 6]
ml_car = 2
ml_truck = 1
data_dict = dict()
num_interval = nb.config.config_dict['DTA']['max_interval']
true_car_x = np.random.rand(num_interval * len(observed_link_list)) * 100
true_truck_x = np.random.rand(num_interval * len(observed_link_list)) * 10
L_car_one = np.random.randint(2, size = (ml_car, len(observed_link_list)))
L_truck_one = np.random.randint(2, size = (ml_truck, len(observed_link_list)))
L_car = csr_matrix(scipy.linalg.block_diag(*[L_car_one for i in range(num_interval)]))
L_truck = csr_matrix(scipy.linalg.block_diag(*[L_truck_one for i in range(num_interval)]))
m_car = L_car.dot(true_car_x)
m_truck = L_truck.dot(true_truck_x)
data_dict['car_count_agg_L_list'] = [L_car]
data_dict['truck_count_agg_L_list'] = [L_truck]
data_dict['car_link_flow'] = [m_car]
data_dict['truck_link_flow'] = [m_truck]
# data_dict['car_link_tt'] = [m_spd_car]
# data_dict['truck_link_tt'] = [m_spd_truck]

In [8]:
config = dict()
config['use_car_link_flow'] = True
config['use_truck_link_flow'] = True
config['use_car_link_tt'] = False
config['use_truck_link_tt'] = False
config['car_count_agg'] = True
config['truck_count_agg'] = True
config['link_car_flow_weight'] = 1
config['link_truck_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 1
config['observed_links'] = observed_link_list
config['paths_list'] = range(nb.config.config_dict['FIXED']['num_path'])

In [9]:
nb.config

[DTA]
network_name = Snap_graph
unit_time = 5
total_interval = -1
assign_frq = 180
start_assign_interval = 0
max_interval = 10
flow_scalar = 10
num_of_link = 7
num_of_node = 6
num_of_O = 1
num_of_D = 1
OD_pair = 1
routing_type = Fixed

[STAT]
rec_mode = LRn
rec_mode_para = 12
rec_folder = record
rec_volume = 0
volume_load_automatic_rec = 0
volume_record_automatic_rec = 0
rec_tt = 1
tt_load_automatic_rec = 0
tt_record_automatic_rec = 0

[FIXED]
path_file_name = path_table
num_path = 3
choice_portion = Buffer
route_frq = 180

In [10]:
dode = MCDODE(nb, config)

In [11]:
dode.add_data(data_dict)

In [12]:
(car_flow, truck_flow) = dode.init_path_flow(car_scale = 10, truck_scale = 1)

In [14]:
dode.estimate_path_flow(max_epoch = 10, car_init_scale = 100, truck_init_scale = 10, store_folder = '.')

Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 0 Loss: 252.18388177952548
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 1 Loss: 185.7179129307376
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 2 Loss: 154.78958002206863
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 3 Loss: 131.97908765908747
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 4 Loss: 119.4839582951794
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 5 Loss: 112.05666301983102
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 6 Loss: 104.01333066240905
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 7 Loss: 95.89158922699828
Running simulation
Getting DAR
Evaluating grad
car link flow
Getting Loss
Epoch: 8 Loss: 91.8957772350777
Running simulation
Getting DAR
Ev

(array([27.20174217, 27.54205355, 39.77528564, 40.59657393, 22.76426227,
         7.51970728, 74.89404074,  7.02409473, 37.36884541, 61.92715986,
        85.43759953, 34.62954087, 91.40987191, 43.56285872, 62.07660118,
        43.30848716, 43.50216905, 93.47949191, 22.43851039, 61.73419499,
        77.04859819, 81.32902487, 25.89185514, 20.10636637, 49.22790146,
        86.49968646, -0.73009373, 63.70908394, 77.88247688, 11.40076905]),
 array([ 4.27655465,  3.02252155,  9.95752756,  1.09846818,  5.5960391 ,
         6.34331743,  3.37305355,  5.82970791,  8.54937444,  1.23200921,
         8.46928265,  7.21486114,  3.13081584,  0.3972414 ,  8.04607899,
         8.27360553,  2.92554652,  2.74068411,  6.90166027,  2.27608722,
         8.46846067,  2.62379689,  6.92671041,  9.31986087,  6.13315411,
        -3.3591045 ,  8.33561803,  1.30007946,  5.5575077 ,  2.08305811]))

In [15]:
from scipy.sparse import coo_matrix

In [19]:
s = np.int(1e8)
p = np.random.rand(s)
coo_matrix((p, (range(s), range(s))))

<100000000x100000000 sparse matrix of type '<type 'numpy.float64'>'
	with 100000000 stored elements in COOrdinate format>

In [18]:
np.random.rand(s)

TypeError: 'float' object cannot be interpreted as an index